<a href="https://colab.research.google.com/github/Geovanecost/TrabalhoN1_Tema02-T-tulo-Gest-o-de-Planejamento-e-dimensionamento-de-esta-es-de-coleta-de-RS/blob/main/Trabalho_N1_Semin%C3%A1rio_de_Algor%C3%ADtmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import math

# PARTE 1 - SOLICITAÇÃO DE DADOS PRIMÁRIOS AO USUÁRIO _______________________________________________________________________________

# Solicição ao usuário com relação ao nome da cidade, ao total de resíduos produzidos
# por kg/mês na cidade e à quantidade de bairros
cidade = input('Informe o nome da cidade: ')
qnt_rs_cidade = float(input('Informe a quantidade, em kg, de resíduos gerados por mês na cidade: '))
n_bairros = int(input('Informe o número de bairros que a cidade possui: '))

nomes_bairros = []  # Lista para armazenar os nomes dos bairros
n_hab_bairro = []  # Lista para armazenar o número de habitantes de cada bairro

# Solicitação ao usuário com relação os nomes dos bairros e inserir na lista
for i in range(n_bairros):  # repetição considerando a quantidade informada pelo usuário
    bairro = input(f'Digite o nome do {i+1}º bairro: ')
    nomes_bairros.append(bairro)

# Solicitação ao usuário com relação ao número de habitantes de cada bairro e barrar
#valor inválido
for bairro in nomes_bairros:
    while True:
        n_hab = int(input(f'Digite o número de habitantes do bairro {bairro}: '))
        if n_hab < 0:
            print('Por favor, insira um número maior ou igual a zero para o número de habitantes.')
        else:
            n_hab_bairro.append(n_hab)
            break
# ________________________________________________________________________________

print('=-=' * 30)
print()
print('::::::::::::::::: Dimensionamento das estações de coleta a serem construídas na cidade :::::::::::::::::::')
print()
print('=-=' * 30)
print()

#_____________________________________________________________________________________
# PARTE 2 - CALCULANDO A QUANTIDADE DE RESÍDUOS SÓLIDOS GERADOS EM CADA BAIRRO POR SEMANA__________________________________________________

# Total de habitantes na cidade
total_habitantes = sum(n_hab_bairro)

residuos_por_bairro_semanal=[]

for i, bairro in enumerate(nomes_bairros):
  # Total de habitantes no bairro atual
  n_hab = n_hab_bairro[i]

    # Quantidade de resíduos gerados semanalmente por um habitante
  residuos_por_habitante_semanal = (qnt_rs_cidade / 4) / total_habitantes

    # Calculo da quantidade de resíduos para o bairro
  residuos_bairro_semanal = round(n_hab * residuos_por_habitante_semanal)

  residuos_por_bairro_semanal.append(residuos_bairro_semanal)

  # PARTE 3 - DIMENSIONAMENTOS _________________________________________________________________________________________________________________

  # Dimensões máximas da estação de coleta

  #larg_max_estacao = 20 metros
  #compri_max_estacao = 30 metros
  #altura_estacao = 7 metros #fixo

  # Volume necessário por m³ para armazenar os resíduos nos depósitos:
  vol_residuos = []
  peso_por_metro_cubico_necessario = 450 # quantidade de kg que corresponde a 1 m³
  for i, residuos_bairro_semanal in enumerate(residuos_por_bairro_semanal):
    voltot_necessario = residuos_bairro_semanal / peso_por_metro_cubico_necessario
    vol_residuos.append(voltot_necessario) # volume necessario no deposito para os resíduos

  # dimensões máximas para cada depósito

  larg_dep_max = 10
  compri_dep_max = 20
  areabase_dep_max = 10 * 20
  proporcao_dep = 1/2 # proporção de 1 para 2 entre largura e comprimento

  #_____________________________________________________________________________________________

  # Listas - dimensões de depósito
  larguras_dep_por_bairro = []
  comprimentos_dep_por_bairro = []
  areas_dep_por_bairro = []

  # Listas - dimensões do setor de triagem
  larguras_triagem_por_bairro = []
  areas_triagem_por_bairro = []

  # Listas - dimensões da estação de coleta
  larguras_estacao_por_bairro = []
  comprimentos_estacao_por_bairro = []
  areas_estacao_por_bairro = []

  # Listas - quantidadae de estações / quantidade de caminhões / quantidade de funcionários
  quantidades_estacoes_por_bairro = []
  funcionarios_estacao_por_bairro = []
  quantidades_caminhao_por_bairro = []

  #_________________________________________________________________________________________________

  for i, voltot_necessario in enumerate(vol_residuos):

    # Dimensões dos depósitos:

    larg_dep = min(larg_dep_max, (areabase_dep_max * proporcao_dep)**0.5)
    # Largura de cada depósito
    larg_dep_total = larg_dep + ((20 * larg_dep) /100) # essa porcentagem é onde os trabalhadores irão circular
    larguras_dep_por_bairro.append(larg_dep_total) # conj das larguras dos depósito de todos os bairro
    # Comprimento de cada depósito
    compri_dep = min(compri_dep_max, larg_dep / proporcao_dep)
    comprimentos_dep_por_bairro.append(compri_dep) # conj dos comprimentos dos depósito de todos os bairros
    # Área de cada depósito
    area_dep_total = larg_dep_total * compri_dep
    areas_dep_por_bairro.append(area_dep_total)

    # Definição da quantidade de estações


    cont = 0

    while True:
      if larg_dep_total >= 10 :
        larg_rest = larg_dep_total - 10
        cont += 1
      if larg_rest < 10:
        cont += 1
        break
    quant_estacao = cont
    quantidades_estacoes_por_bairro.append(quant_estacao)

    # Dimensões do setor de triagem

    larg_triagem = 2 * larg_dep_total  # metros (largura dos dois depósitos juntos)
    larguras_triagem_por_bairro.append(larg_triagem)
    compri_triagem = 10  # metros
    area_triagem = larg_triagem * compri_triagem # metros quadrados
    areas_triagem_por_bairro.append(area_triagem)

  # Dimesões da estação de coleta
    larg_estacao = 2 * larg_dep_total
    larguras_estacao_por_bairro.append(larg_estacao)
    compri_estacao = compri_dep + compri_triagem
    comprimentos_estacao_por_bairro.append(compri_estacao)
    area_estacao = larg_estacao * compri_estacao
    areas_estacao_por_bairro.append(area_estacao)

    # Calcular a área efetiva considerando a proporção 1:2
    # Se a largura é x metros, então o comprimento será 2𝑥 metros. Portanto, a área efetiva será 2 * (largura)².
    area_estacao = 2 * (larg_estacao ** 2)
    areas_estacao_por_bairro.append(area_estacao)
    altura = 7 # metros
  #________________________________________________________________________________________________________________________
  # PARTE 4 - NÚMERO DE TRABALHADORES E CAMINHÕES PARA A(S) ESTAÇÕES DE COLETA

  # NUMERO DE TRABALHADORES

    funcionarios = area_estacao / 2 # 1 trabalhadores por 2 metros quadrados
    funcionarios_estacao_por_bairro.append(funcionarios)

  # NUMERO DE CAMINHÕES DESCARREGANDO OS RESÍDUOS PRÉ TRIADOS E LEVANDO OS RESÍDUOS TRIADOS

    quant_caminhao = 0
    caminhao_kg = 70000 # quantidade que um caminhão consegue carregar
    for residuos_bairro_semanal in range (len(residuos_por_bairro_semanal)):
      quant = residuos_bairro_semanal - caminhao_kg
      if quant <= 0: # se o valor da quantidade de residuos semanal por bairro for menor ou igual que a capacidade do caminhão será apenas um recolhendo
        quant_caminhao = 1
      elif quant > 0: # se o valor da quantidade de residuos semanal por bairro for maior que a capacidade do caminhão serão mais que um recolhendo
        while residuos_bairro_semanal > caminhao_kg:
          residuos_rest = residuos_bairro_semanal - caminhao_kg
          quant_caminhao += 1
          if residuos_bairro_semanal < caminhao_kg:
            quant_caminhao += 1
            break
      quant_caminhao_total_bairro = quant_caminhao
      quantidades_caminhao_por_bairro.append(quant_caminhao_total_bairro)

  #____________________________________________________________________________________________________________________________

  # PARTE 5 - IMPRIMINDO RESULTADOS

  # Informações por bairro da cidade: Dimensionamento da estação, dos depósitos, do setor de triagem e a quantidade de estações de coletas:
  for i, residuos_bairro_semanal in enumerate(residuos_por_bairro_semanal):
   print(f'Residuos semanais por bairro : {residuos_por_bairro_semanal[i]}')
  for i, bairro in enumerate(nomes_bairros):
    print(f'Bairro: {bairro[i]}')
    print(f'Número de habitantes: {n_hab_bairro[i]}')

    print()
    print('Dimensões de cada depósito da(s) Estações(s):') #_____________________________________
    print()
    for i, larg_dep_total in enumerate(larguras_dep_por_bairro):
      print(f'Largura: {larg_dep_total} m')
    for i, compri_dep in enumerate(comprimentos_dep_por_bairro):
      print(f'Comprimento: {compri_dep} m')
    for i, area_dep_total in enumerate(areas_dep_por_bairro):
      print(f'Área: {area_dep_total} m²')
      print()

    print('Dimensões do Setor de Triagem: ') #___________________________________________________
    print()
    for i, larg_triagem in enumerate(larguras_triagem_por_bairro):
      print(f'Largura: {larg_triagem} m')
      print(f'Comprimento: 10 m')
    for i, area_triagem in enumerate(areas_triagem_por_bairro):
      print(f'Área: {area_triagem} m²')
    print()
    print('Dimensões da Estação de coleta e Triagem:') #________________________________________
    print()
    for i, larg_estacao in enumerate(larguras_estacao_por_bairro):
      print(f'Largura: {larg_estacao} m')
    for i, compri_estacao in enumerate(comprimentos_estacao_por_bairro):
      print(f'Comprimento: {compri_estacao} m')
    for i, area_estacao in enumerate(areas_estacao_por_bairro) :
      print(f'Área: {area_estacao} m²')
    print('Altura: 7 m')
    print()

    for i, quant_estacao in enumerate(quantidades_estacoes_por_bairro):
      print(f'Quantidade de Estações de Coleta e Triagem necessária para o bairro: {quant_estacao} unidades')
    print()
    print('=-=' * 30)
    print()
    print('Dados  importantes:') #_______________________________________________________________
    print()
    # Quantidade de funcionários necessários:
    for i, funcionarios in enumerate(funcionarios_estacao_por_bairro):
      print(f'Quantidade de funcionários necessários por Estação: {funcionarios}')
    print()
    # Quantidade de caminhões de coleta necessários:
    for i, quant_caminhao_total_bairro in enumerate(quantidades_caminhao_por_bairro):
      print(f'Quantidade necessária de caminhões de coleta: {quant_caminhao_total_bairro}')
    print()
    print('=-=' * 30)



Informe o nome da cidade: natal
Informe a quantidade, em kg, de resíduos gerados por mês na cidade: 20000
Informe o número de bairros que a cidade possui: 2
Digite o nome do 1º bairro: asdasf
Digite o nome do 2º bairro: fasgs
Digite o número de habitantes do bairro asdasf: 25564
Digite o número de habitantes do bairro fasgs: 5652
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

::::::::::::::::: Dimensionamento das estações de coleta a serem construídas na cidade :::::::::::::::::::

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

Residuos semanais por bairro : 4095
Bairro: a
Número de habitantes: 25564

Dimensões de cada depósito da(s) Estações(s):

Largura: 12.0 m
Comprimento: 20 m
Área: 240.0 m²

Dimensões do Setor de Triagem: 

Largura: 24.0 m
Comprimento: 10 m
Área: 240.0 m²

Dimensões da Estação de coleta e Triagem:

Largura: 24.0 m
Comprimento: 30 m
Área: 720.0 m²
Área: 1152.0 m²
Altura: 7 m



In [3]:
import math

# PARTE 1 - SOLICITAÇÃO DE DADOS PRIMÁRIOS AO USUÁRIO ______________________________________________________

# Solicitação ao usuário com relação ao nome da cidade, ao total de resíduos produzidos
# por kg/mês na cidade e à quantidade de bairros
cidade = input('Informe o nome da cidade: ')
qnt_rs_cidade = float(input('Informe a quantidade, em kg, de resíduos gerados por mês na cidade: '))
n_bairros = int(input('Informe o número de bairros que a cidade possui: '))

nomes_bairros = []  # Lista para armazenar os nomes dos bairros
n_hab_bairro = []  # Lista para armazenar o número de habitantes de cada bairro

# Solicitação ao usuário com relação os nomes dos bairros e inserir na lista
for i in range(n_bairros):
    bairro = input(f'Digite o nome do {i+1}º bairro: ')
    nomes_bairros.append(bairro)

# Solicitação ao usuário com relação ao número de habitantes de cada bairro
for bairro in nomes_bairros:
    while True:
        n_hab = int(input(f'Digite o número de habitantes do bairro {bairro}: '))
        if n_hab < 0:
            print('Por favor, insira um número maior ou igual a zero para o número de habitantes.')
        else:
            n_hab_bairro.append(n_hab)
            break

print('=-=' * 30)
print()
print('::::::::::::::::: Dimensionamento das estações de coleta a serem construídas na cidade :::::::::::::::::::')
print()
print('=-=' * 30)
print()

# PARTE 2 - CALCULANDO A QUANTIDADE DE RESÍDUOS SÓLIDOS GERADOS EM CADA BAIRRO POR SEMANA _____________________

# Total de habitantes na cidade
total_habitantes = sum(n_hab_bairro)

residuos_por_bairro_semanal = []

for i, bairro in enumerate(nomes_bairros):
    # Quantidade de resíduos gerados semanalmente por um habitante
    residuos_por_habitante_semanal = (qnt_rs_cidade / 4) / total_habitantes

    # Calculo da quantidade de resíduos para o bairro
    residuos_bairro_semanal = round(n_hab_bairro[i] * residuos_por_habitante_semanal)
    residuos_por_bairro_semanal.append(residuos_bairro_semanal)

    # PARTE 3 - DIMENSIONAMENTOS _________________________________________________________________________________

    # Cálculos para dimensões de depósitos, triagem, etc.
    # ...

    # Aqui dentro você realiza os cálculos e armazena em listas como larguras_dep_por_bairro, areas_triagem_por_bairro, etc.

# PARTE 4 - IMPRIMINDO RESULTADOS _____________________________________________________________________________

# Informações por bairro da cidade: Dimensionamento da estação, dos depósitos, do setor de triagem e a quantidade de estações de coletas:
for i, bairro in enumerate(nomes_bairros):
    print(f'Residuos semanais por bairro : {residuos_por_bairro_semanal[i]}')
    print(f'Bairro: {bairro}')
    print(f'Número de habitantes: {n_hab_bairro[i]}')
    print()
    print('Dimensões de cada depósito da(s) Estações(s):')
    print()
    print(f'Largura: {larguras_dep_por_bairro[i]} m')
    print(f'Comprimento: {comprimentos_dep_por_bairro[i]} m')
    print(f'Área: {areas_dep_por_bairro[i]} m²')
    print()
    print('Dimensões do Setor de Triagem:')
    print()
    print(f'Largura: {larguras_triagem_por_bairro[i]} m')
    print(f'Comprimento: 10 m')
    print(f'Área: {areas_triagem_por_bairro[i]} m²')
    print()
    print('Dimensões da Estação de coleta e Triagem:')
    print()
    print(f'Largura: {larguras_estacao_por_bairro[i]} m')
    print(f'Comprimento: {comprimentos_estacao_por_bairro[i]} m')
    print(f'Área: {areas_estacao_por_bairro[i]} m²')
    print(f'Altura: 7 m')
    print()
    print(f'Quantidade de Estações de Coleta e Triagem necessária para o bairro: {quantidades_estacoes_por_bairro[i]} unidades')
    print()
    print('=-=' * 30)
    print()
    print('Dados importantes:')
    print()
    print(f'Quantidade de funcionários necessários por Estação: {funcionarios_estacao_por_bairro[i]}')
    print(f'Quantidade necessária de caminhões de coleta: {quantidades_caminhao_por_bairro[i]}')
    print()
    print('=-=' * 30)
    print()


Informe o nome da cidade: natal
Informe a quantidade, em kg, de resíduos gerados por mês na cidade: 2000
Informe o número de bairros que a cidade possui: 2
Digite o nome do 1º bairro: nkghj
Digite o nome do 2º bairro: çjb,n
Digite o número de habitantes do bairro nkghj: 25654
Digite o número de habitantes do bairro çjb,n: 2654
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

::::::::::::::::: Dimensionamento das estações de coleta a serem construídas na cidade :::::::::::::::::::

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

Residuos semanais por bairro : 453
Bairro: nkghj
Número de habitantes: 25654

Dimensões de cada depósito da(s) Estações(s):

Largura: 12.0 m
Comprimento: 20 m
Área: 240.0 m²

Dimensões do Setor de Triagem:

Largura: 24.0 m
Comprimento: 10 m
Área: 240.0 m²

Dimensões da Estação de coleta e Triagem:

Largura: 24.0 m
Comprimento: 30 m
Área: 720.0 m²
Altura: 7 m

Quantidade de Est

In [7]:
import math

# PARTE 1 - SOLICITAÇÃO DE DADOS PRIMÁRIOS AO USUÁRIO ______________________________________________________

# Solicitação ao usuário com relação ao nome da cidade, ao total de resíduos produzidos
# por kg/mês na cidade e à quantidade de bairros
cidade = input('Informe o nome da cidade: ')
qnt_rs_cidade = float(input('Informe a quantidade, em kg, de resíduos gerados por mês na cidade: '))
n_bairros = int(input('Informe o número de bairros que a cidade possui: '))

nomes_bairros = []  # Lista para armazenar os nomes dos bairros
n_hab_bairro = []  # Lista para armazenar o número de habitantes de cada bairro

# Solicitação ao usuário com relação os nomes dos bairros e inserir na lista
for i in range(n_bairros):
    bairro = input(f'Digite o nome do {i+1}º bairro: ')
    nomes_bairros.append(bairro)

# Solicitação ao usuário com relação ao número de habitantes de cada bairro
for bairro in nomes_bairros:
    while True:
        n_hab = int(input(f'Digite o número de habitantes do bairro {bairro}: '))
        if n_hab < 0:
            print('Por favor, insira um número maior ou igual a zero para o número de habitantes.')
        else:
            n_hab_bairro.append(n_hab)
            break

print('=-=' * 30)
print()
print('::::::::::::::::: Dimensionamento das estações de coleta a serem construídas na cidade :::::::::::::::::::')
print()
print('=-=' * 30)
print()

# PARTE 2 - CALCULANDO A QUANTIDADE DE RESÍDUOS SÓLIDOS GERADOS EM CADA BAIRRO POR SEMANA _____________________

# Total de habitantes na cidade
total_habitantes = sum(n_hab_bairro)

residuos_por_bairro_semanal = []

for i, bairro in enumerate(nomes_bairros):
    # Quantidade de resíduos gerados semanalmente por um habitante
    residuos_por_habitante_semanal = (qnt_rs_cidade / 4) / total_habitantes

    # Calculo da quantidade de resíduos para o bairro
    residuos_bairro_semanal = round(n_hab_bairro[i] * residuos_por_habitante_semanal)
    residuos_por_bairro_semanal.append(residuos_bairro_semanal)

    # PARTE 3 - DIMENSIONAMENTOS _________________________________________________________________________________

    # Simulação das dimensões dos depósitos, setor de triagem e estação de coleta (substitua pelos cálculos reais)
    larguras_dep_por_bairro = [10.5, 12.0, 11.0]  # Exemplo, substitua pelos cálculos reais
    comprimentos_dep_por_bairro = [15.0, 18.0, 16.5]  # Exemplo, substitua pelos cálculos reais
    areas_dep_por_bairro = [larguras_dep_por_bairro[i] * comprimentos_dep_por_bairro[i] for i in range(len(nomes_bairros))]

    larguras_triagem_por_bairro = [7.5, 8.0, 8.5]  # Exemplo, substitua pelos cálculos reais
    areas_triagem_por_bairro = [larguras_triagem_por_bairro[i] * 10.0 for i in range(len(nomes_bairros))]

    larguras_estacao_por_bairro = [13.0, 14.0, 12.0]  # Exemplo, substitua pelos cálculos reais
    comprimentos_estacao_por_bairro = [20.0, 22.0, 18.0]  # Exemplo, substitua pelos cálculos reais
    areas_estacao_por_bairro = [larguras_estacao_por_bairro[i] * comprimentos_estacao_por_bairro[i] for i in range(len(nomes_bairros))]

    quantidades_estacoes_por_bairro = [3, 4, 3]  # Exemplo, substitua pelos cálculos reais
    funcionarios_estacao_por_bairro = [8, 9, 7]  # Exemplo, substitua pelos cálculos reais
    quantidades_caminhao_por_bairro = [5, 6, 4]  # Exemplo, substitua pelos cálculos reais

# PARTE 4 - IMPRIMINDO RESULTADOS _____________________________________________________________________________

# Informações por bairro da cidade: Dimensionamento da estação, dos depósitos, do setor de triagem e a quantidade de estações de coletas:
for i, bairro in enumerate(nomes_bairros):
    print(f'Residuos semanais por bairro : {residuos_por_bairro_semanal[i]}')
    print(f'Bairro: {bairro}')
    print(f'Número de habitantes: {n_hab_bairro[i]}')
    print()
    print('Dimensões de cada depósito da(s) Estações(s):')
    print()
    print(f'Largura: {larguras_dep_por_bairro[i]} m')
    print(f'Comprimento: {comprimentos_dep_por_bairro[i]} m')
    print(f'Área: {areas_dep_por_bairro[i]} m²')
    print()
    print('Dimensões do Setor de Triagem:')
    print()
    print(f'Largura: {larguras_triagem_por_bairro[i]} m')
    print(f'Comprimento: 10 m')
    print(f'Área: {areas_triagem_por_bairro[i]} m²')
    print()
    print('Dimensões da Estação de coleta e Triagem:')
    print()
    print(f'Largura: {larguras_estacao_por_bairro[i]} m')
    print(f'Comprimento: {comprimentos_estacao_por_bairro[i]} m')
    print(f'Área: {areas_estacao_por_bairro[i]} m²')
    print(f'Altura: 7 m')
    print()
    print(f'Quantidade de Estações de Coleta e Triagem necessária para o bairro: {quantidades_estacoes_por_bairro[i]} unidades')
    print()
    print('=-=' * 30)
    print()
    print('Dados importantes:')
    print()
    print(f'Quantidade de funcionários necessários por Estação: {funcionarios_estacao_por_bairro[i]}')
    print(f'Quantidade necessária de caminhões de coleta: {quantidades_caminhao_por_bairro[i]}')
    print()
    print('=-=' * 30)
    print()


Informe o nome da cidade: natal
Informe a quantidade, em kg, de resíduos gerados por mês na cidade: 556544
Informe o número de bairros que a cidade possui: 2
Digite o nome do 1º bairro: asafa
Digite o nome do 2º bairro: gasg
Digite o número de habitantes do bairro asafa: 2000
Digite o número de habitantes do bairro gasg: 353563
=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

::::::::::::::::: Dimensionamento das estações de coleta a serem construídas na cidade :::::::::::::::::::

=-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-==-=

Residuos semanais por bairro : 783
Bairro: asafa
Número de habitantes: 2000

Dimensões de cada depósito da(s) Estações(s):

Largura: 10.5 m
Comprimento: 15.0 m
Área: 157.5 m²

Dimensões do Setor de Triagem:

Largura: 7.5 m
Comprimento: 10 m
Área: 75.0 m²

Dimensões da Estação de coleta e Triagem:

Largura: 13.0 m
Comprimento: 20.0 m
Área: 260.0 m²
Altura: 7 m

Quantidade de E